In [78]:
import numpy as np
import pandas as pd
import datetime as dt

In [79]:
df = pd.read_csv('OnlineRetail.csv' , sep=';')

In [80]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Al ejecutar tu análisis exploratorio, describe el número de columnas y filas que encuentras, en cuáles encuentras valores nulos o perdidos y cuántos.
para saber el numero de columnas y filas usaremos la funcion df.shape[]

numero de columnas:

In [81]:
df.shape[1]

8

numero de filas:

In [82]:
df.shape[0]

541909

¿Cómo podemos saber si hay valores nulos en alguna columna para poder procesar esos datos? 

In [83]:
df.isna().any()

InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
dtype: bool

para saber cuantos valores perdidos hay en cada columna usaremos la funcion df.isnull().sum()

In [84]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

como si encontramos valores nulos los eliminaremos utilizando la funcion df.dropna()

In [85]:
df = df.dropna()

Calcula la media, máximo, mínimo y desviaciónestándar de las columnas con valores numéricos para ello utilizaremos la funcion df.describe() que nos arrojan estos datos y mas

In [86]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


Si encuentras un valor que no tenga lógica, es decir valores negativos en columnas donde no deberían haber, elimínalos. para ello utilize la funcion df.drop con una condicion

In [87]:
df = df.drop(df[df['Quantity']<=0].index)

In [88]:
df = df.drop(df[df['UnitPrice']<=0].index)

In [89]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,397884.000000,397884.000000,397884.000000
mean,12.988238,3.116488,15294.423453
std,179.331775,22.097877,1713.141560
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


Calcula cuántos valores únicos encontraste dentro de cada columna después de limpiar los datos.
para ello utilizare la funcion df.nunique()

In [90]:
df.nunique()

InvoiceNo      18532
StockCode       3665
Description     3877
Quantity         301
InvoiceDate    17282
UnitPrice        440
CustomerID      4338
Country           37
dtype: int64

Calcula la Recency (Reciente), Frequency (Frecuencia) y Monetary value (Valor monetario)

Calcula la Recency (Reciente)



In [91]:
df.loc[:,['InvoiceDate']] = pd.to_datetime(df['InvoiceDate'])

In [92]:
df_recency = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max()
df_recency.columns = ['CustomerID', 'LastDate']
recent_date = df_recency['LastDate'].max()
df_recency['Recency'] = df_recency['LastDate'].apply(lambda x: (recent_date - x).days)
df_recency.head()

,CustomerID,LastDate,Recency
0,12346.0,2011-01-18 10:01:00,325
1,12347.0,2011-12-07 15:52:00,1
2,12348.0,2011-09-25 13:13:00,74
3,12349.0,2011-11-21 09:51:00,18
4,12350.0,2011-02-02 16:01:00,309


Frequency (Frecuencia)

In [93]:
df_frequency = df.drop_duplicates().groupby(by='CustomerID', as_index=False)['InvoiceDate'].count()
df_frequency.columns = ['CustomerID', 'Frequency']
df_frequency.head()

,CustomerID,Frequency
0,12346.0,1
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


Monetary value (Valor monetario)

In [96]:
df['total'] = df['Quantity']*df['UnitPrice']

In [97]:
df_monetary = df.groupby(by='CustomerID', as_index=False)['total'].sum()
df_monetary.columns = ['CustomerID', 'Monetary']
print(df_monetary)

      CustomerID  Monetary
0        12346.0  77183.60
1        12347.0   4310.00
2        12348.0   1797.24
3        12349.0   1757.55
4        12350.0    334.40
...          ...       ...
4333     18280.0    180.60
4334     18281.0     80.82
4335     18282.0    178.05
4336     18283.0   2094.88
4337     18287.0   1837.28

[4338 rows x 2 columns]


Incluye estos valores en una tabla

In [98]:
new2 = pd.DataFrame([df_monetary.CustomerID,
                    df_recency.Recency,
                    df_frequency.Frequency,
                    df_monetary.Monetary]).transpose()

In [99]:
new2.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325.0,1.0,77183.60
1,12347.0,1.0,182.0,4310.00
2,12348.0,74.0,31.0,1797.24
3,12349.0,18.0,73.0,1757.55
4,12350.0,309.0,17.0,334.40


Calcula los puntajes o scores con un rango de 1 a 5

In [74]:
quantiles = new2.quantile(q=[0.2,0.4,0.6,0.8])
quantiles = quantiles.to_dict()

In [100]:
segmented_rfm = new2

In [101]:
def RScore(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 4
    else:
        return 5
    
def FMScore(x,p,d):
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 2
    else:
        return 1

In [102]:
new2['r_quartile'] = new2['Recency'].apply(RScore, args=('Recency',quantiles,)) 
segmented_rfm['f_quartile'] = segmented_rfm['Frequency'].apply(FMScore, args=('Frequency',quantiles,)) 
segmented_rfm['m_quartile'] = segmented_rfm['Monetary'].apply(FMScore, args=('Monetary',quantiles,)) 
segmented_rfm.head()

,CustomerID,Recency,Frequency,Monetary,r_quartile,f_quartile,m_quartile
0,12346.0,325.0,1.0,77183.60,5,5,1
1,12347.0,1.0,182.0,4310.00,1,1,1
2,12348.0,74.0,31.0,1797.24,4,3,2
3,12349.0,18.0,73.0,1757.55,2,2,2
4,12350.0,309.0,17.0,334.40,5,4,4
